# Results Notebook

In [26]:
%load_ext autoreload
%autoreload 2

import hwacctools.quantization.quant as q
from stim_lib.stimulus_gen import generate_qracc_inputs
import numpy as np
import plotly.express as px
import seaborn as sns
import pandas as pd
import plotly.subplots as sp

df = pd.DataFrame
sns.set_theme()

def melt_row_col_data(arr,name):
    arr_df = df(arr)
    arr_df = arr_df.melt(
        ignore_index=False,
        var_name='Column',
        value_name='ADC_OUT_AMS',
    )
    arr_df['Row'] = arr_df.index
    return arr_df

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [27]:
# Generate the reference result

w,x,wx_outBits = generate_qracc_inputs(
    wDimX = 32, #nColumns
    wDimY = 128, #nRows
    xBatches = 10,
    xTrits = 1,
    outBits = 4,
    seed = 0,
    weight_mode = 'bipolar',
    col_symmetric=False
)
wx = w @ x.T
wx_rounded = np.round(wx.T / 2**2).astype(int).T[::-1].T

[STIM_GEN] Generating random weights


### Comparing AMS vs RTL simulations of QRAcc

In [28]:
ams = np.loadtxt('../tb/qracc/inputs/adc_out.txt')
rtl = np.loadtxt('../tb/qracc/inputs/adc_out_rtlsims.txt')

ams_df = df(ams)
rtl_df = df(rtl)

# ams_df['Row'] = ams_df.index
# rtl_df['Row'] = rtl_df.index

ams_df = ams_df.melt(
    ignore_index=False,
    var_name='Column',
    value_name='ADC_OUT_AMS',
)
rtl_df = rtl_df.melt(
    ignore_index=False,
    var_name='Column',
    value_name='ADC_OUT_RTL',
)
ams_df['Row'] = ams_df.index
rtl_df['Row'] = rtl_df.index
# Join the two dataframes
adc_df = pd.merge(ams_df, rtl_df)

rtl_fp = rtl.flatten()/2**3
ams_fp = ams.flatten()/2**3

err =   rtl_fp - ams_fp
snr = 20*np.log(np.var(rtl_fp) / np.var(err))
var_err = np.var(err)
var_rtl = np.var(rtl_fp)

rmse = np.sqrt(np.mean((rtl.flatten() - ams.flatten())**2))
print(f'AMS vs Shifted, RMSE={rmse:.2f}, SNR={snr:.2f}dB, var_err={var_err:.2e}, var_yo={var_rtl:.2e}')
fig = px.scatter(adc_df, x='ADC_OUT_AMS', y='ADC_OUT_RTL', width=400, height=400, title=f'QRacc AMS vs Shifted, RMSE={rmse:.2f}, SNR={snr:.2f}dB, Var_err={var_err:.2f}, Var_rtl={var_rtl:.2f}', labels={'ADC_OUT_AMS':'AMS','ADC_OUT_RTL':'RTL'})
fig.show()
fig.write_image('../images/rtl_vs_ams.svg')

AMS vs Shifted, RMSE=0.00, SNR=infdB, var_err=0.00e+00, var_yo=7.86e-02


/tmp/ipykernel_58535/1592600238.py:29: RuntimeWarning:

divide by zero encountered in scalar divide



# Sequential Accelerator Simulations

In [29]:
w,x,wx_outBits = generate_qracc_inputs(
    wDimX = 32, #nColumns
    wDimY = 128, #nRows
    xBatches = 10,
    xTrits = 3,
    outBits = 4,
    seed = 0,
    weight_mode = 'bipolar',
    col_symmetric=False
)

# ams = np.loadtxt('../tb/seq_acc/inputs/adc_out.txt')
rtl = np.loadtxt('../tb/seq_acc/inputs/mac_out_rtlsims.txt')

wx = w @ x.T

# rtl.astype(int), wx[::-1].T

rtl_fp = rtl.flatten()#/(2**8)
wx_fp = wx[::-1].T.flatten()#/(2**8)

rmse = np.sqrt(np.mean((rtl_fp - wx_fp)**2))
# SNR

err = rtl_fp - wx_fp
snr = 20*np.log(np.var(rtl_fp) / np.var(err))
var_err = np.var(err)
var_rtl = np.var(rtl_fp)

a = px.scatter(
    x=rtl.flatten(), 
    y=wx[::-1].T.flatten(), 
    width=400, 
    height=400, 
    title=f'Ideal vs RTL, RMSE={rmse:.2f}, SNR={snr:.2f}dB', 
    labels={'x': 'RTL', 'y': 'Ideal'}
)
b = px.histogram(
    err,
    width = 400,
    height = 400,
    title = 'Error Histogram',
    nbins=500
)

fig = sp.make_subplots(rows=1, cols=2)
fig.append_trace(a["data"][0], 1, 1)
fig.append_trace(b["data"][0], 1, 2)

fig.update_layout(
    title_text=f'Seq Acc Ideal vs RTL, RMSE={rmse:.2f}, SNR={snr:.2f}dB, var_err={var_err:.2e}, var_yo={var_rtl:.2e}',
    showlegend=False
)
fig.update_xaxes(title_text='RTL', row=1, col=1)
fig.update_yaxes(title_text='Ideal', row=1, col=1)
fig.update_xaxes(title_text='Error', row=1, col=2)
fig.show()
print(f'SNR of ideal ADC: {6.02*4 + 1.76:.2f}dB')

[STIM_GEN] Generating random weights


SNR of ideal ADC: 25.84dB


In [30]:
# q.simulate_trit_mul(w, x[0], trits = 3, verbose=True)

In [31]:
w,x,wx_outBits = generate_qracc_inputs(
    wDimX = 32, #nColumns
    wDimY = 128, #nRows
    xBatches = 10,
    xTrits = 3,
    outBits = 4,
    seed = 0,
    weight_mode = 'bipolar',
    col_symmetric=False
)

# ams = np.loadtxt('../tb/seq_acc/inputs/adc_out.txt')
ams = np.loadtxt('../tb/seq_acc/inputs/mac_out_ams.txt')

wx = w @ x.T

# ams.astype(int), wx[::-1].T

ams_fp = ams.flatten()#/(2**8)
wx_fp = wx[::-1].T.flatten()#/(2**8)

rmse = np.sqrt(np.mean((ams_fp - wx_fp)**2))
# SNR

err = ams_fp - wx_fp
snr = 20*np.log(np.var(ams_fp) / np.var(err))
var_err = np.var(err)
var_rtl = np.var(ams_fp)

a = px.scatter(
    x=ams.flatten(), 
    y=wx[::-1].T.flatten(), 
    width=400, 
    height=400, 
    title=f'Ideal vs RTL, RMSE={rmse:.2f}, SNR={snr:.2f}dB', 
    labels={'x': 'RTL', 'y': 'Ideal'}
)
a.update_traces(marker=dict(size=3))
b = px.histogram(
    err,
    width = 400,
    height = 400,
    title = 'Error Histogram',
    nbins=500
)

fig = sp.make_subplots(rows=1, cols=2)
fig.append_trace(a["data"][0], 1, 1)
fig.append_trace(b["data"][0], 1, 2)

fig.update_layout(
    title_text=f'Seq Acc Ideal vs AMS, RMSE={rmse:.2f}, SNR={snr:.2f}dB, var_err={var_err:.2e}, var_yo={var_rtl:.2e}',
    showlegend=False
)
fig.update_xaxes(title_text='AMS', row=1, col=1)
fig.update_yaxes(title_text='Ideal', row=1, col=1)
fig.update_xaxes(title_text='Error', row=1, col=2)
fig.show()

print(f'SNR of ideal ADC: {6.02*4 + 1.76:.2f}dB')

[STIM_GEN] Generating random weights


SNR of ideal ADC: 25.84dB


In [32]:
df(ams.astype(int))

,0,1,2,3,4,5,6,7,8,9,...,22,23,24,25,26,27,28,29,30,31
0,-20,68,4,0,40,-12,0,0,-44,84,...,0,-16,40,48,40,64,-12,80,-108,32
1,-20,24,36,-44,0,56,60,48,-40,-36,...,36,-32,68,-24,52,-44,16,36,28,-124
2,68,-72,64,44,-24,-28,-20,100,32,28,...,48,-8,-48,88,-40,104,48,24,28,72
3,64,-20,-8,8,48,68,-8,-60,40,20,...,-32,-56,-28,56,-52,-4,92,-8,16,8
4,-20,-32,-60,-12,4,88,44,24,-12,-24,...,48,12,-80,-44,0,-20,104,52,48,56
5,24,-4,-8,4,-36,44,-12,36,-104,-32,...,-20,20,44,-8,-8,-56,-48,32,12,-40
6,84,-36,8,-4,-24,-24,-8,-12,28,12,...,12,-12,52,-84,60,24,28,44,44,96
7,60,-88,-60,44,-40,32,-68,124,68,20,...,56,-56,4,4,64,-16,-4,44,-48,28
8,-104,8,-56,-40,68,-56,20,36,-12,80,...,-36,44,80,-68,-4,-24,28,64,-64,32
9,0,-40,40,4,-40,16,76,-32,-48,-28,...,124,-72,40,-56,88,80,72,24,-20,88


In [33]:
df(wx[::-1].T)

,0,1,2,3,4,5,6,7,8,9,...,22,23,24,25,26,27,28,29,30,31
0,-38,54,-4,-6,30,-24,-8,-18,-60,68,...,-16,-22,22,36,22,58,-24,60,-118,22
1,-28,14,26,-50,-16,36,42,30,-46,-46,...,22,-50,60,-40,36,-64,8,28,8,112
2,48,-80,48,24,-40,-34,-26,96,10,20,...,42,-26,-60,70,-56,92,42,8,18,54
3,61,-29,-27,7,33,55,-19,-77,25,5,...,-51,-83,-43,41,-79,-9,77,-13,11,3
4,-28,-52,-78,-24,-8,76,28,4,-26,-46,...,34,6,-90,-54,-12,-36,102,32,36,36
5,17,-11,-21,-11,-53,25,-19,19,147,-37,...,-33,13,39,-21,-17,-65,-61,15,3,-53
6,69,-49,-9,-19,-35,-35,-17,-25,11,-1,...,3,-27,43,-91,53,17,17,37,33,83
7,47,-103,-79,27,-57,25,-83,113,45,5,...,45,-77,-15,-9,53,-25,-11,35,-69,13
8,142,4,-68,-64,54,-64,10,22,-28,74,...,-48,30,66,-88,-20,-46,22,54,-74,28
9,-17,-53,21,-9,-51,5,57,-43,-55,-37,...,105,-87,27,-71,75,69,63,5,-41,69
